In [12]:
import requests
import pandas as pd
import re
from datetime import datetime 
# Your Instagram Business Account User ID and long-lived access token

# The hashtag you want to search for (without the '#' symbol)
HASHTAG = 'makaw'  # Replace with your desired hashtag (without the '#')
INSTAGRAM_USER_ID = '17841472712121977'
ACCESS_TOKEN = 'EAANoZCe4Hve0BO1kZAI1Qwibz9l4Eh9sx8jDHZC0LKXtPSlYwXV7f7SZB0r1IAVlraI5FaahbNsIWDolPtbbwyp2rhluqSPox3RLctCx8bRYVxZAGG4SZAcyx40s4sZB3mxQU5aVlpnoOGbyjHyUrXdlleWA1VuPkWOR11OE5nL5ZAIUxHbpi4wd6QZDZD'
# Request URL to get the hashtag ID


In [13]:
def extract_hashtags(caption):
    hashtags = re.findall(r'#\w+', caption)  # Find all words starting with #
    caption_without_hashtags = re.sub(r'#\w+', '', caption).strip()  # Remove hashtags from caption
    return caption_without_hashtags, ' '.join(hashtags)  # Return cleaned caption and joined hashtags


In [14]:
#GET HASHTAG_ID FOR A CERTAIN HASHTAG

url = f"https://graph.facebook.com/v18.0/ig_hashtag_search"
params = {
    "user_id": INSTAGRAM_USER_ID,
    "q": HASHTAG,  # Search for the hashtag
    "access_token": ACCESS_TOKEN
}

# Make the API request
response = requests.get(url, params=params)

# Parse the response JSON to get the hashtag ID
hashtag_data = response.json()

# Extract the hashtag ID (first result)
hashtag_id = hashtag_data.get("data", [{}])[0].get("id", None)

print("Hashtag ID:", hashtag_id)

Hashtag ID: 17841562030128143


In [15]:

# Request URL to get media for the hashtag ID
media_url = f"https://graph.facebook.com/v18.0/{hashtag_id}/recent_media"
params = {
    "user_id": INSTAGRAM_USER_ID,
    "fields": "id,caption,media_type,media_url,permalink,timestamp",  # Removed owner field
    "access_token": ACCESS_TOKEN
}

# Make the API request to get the media
response = requests.get(media_url, params=params)
print(response)
# Check if the response is successful
if response.status_code == 200:
    # Parse the response JSON
    media_data = response.json()

    # Initialize the media list
    media_list = []

    # Loop through each media item in the response data
    for media in media_data.get("data", []):
        if media["media_type"] == "IMAGE":  # Only include images
            caption = media.get("caption", "No caption")
            cleaned_caption, hashtags = extract_hashtags(caption)  # Split caption and hashtags

            post_info = {
                "id": media["id"],
                "caption": cleaned_caption,  # Caption without hashtags
                "hashtags": hashtags,  # Extracted hashtags
                "media_type": media["media_type"],
                "media_url": media.get("media_url", "No URL"),
                "permalink": media["permalink"],
                "timestamp": media["timestamp"],
                "like_count": 0  # Placeholder for like count
            }

        # Now fetch the like count for the specific media ID
        media_id = media['id']
        like_count_url = f"https://graph.facebook.com/v18.0/{media_id}"
        like_count_params = {
            "fields": "like_count",  # Request the like_count field along with the media ID
            "access_token": ACCESS_TOKEN
        }

        # Make the API request to get the like count
        like_response = requests.get(like_count_url, params=like_count_params)

        if like_response.status_code == 200:
            like_data = like_response.json()
            # Update the post_info with the like count
            post_info['like_count'] = like_data.get('like_count', 0)
        else:
            print(f"Error fetching like count for Media ID: {media_id}, Status Code: {like_response.status_code}")

        # Append the structured media information to the media list
        media_list.append(post_info)
        
    api_completed_timestamp = datetime.utcnow().isoformat() 
    # Create a pandas DataFrame from the media list
    media_df = pd.DataFrame(media_list)
    media_df["latest_query_completed_timestamp"] = api_completed_timestamp

    # Display the DataFrame (if in Jupyter, this will render as a nice table)
    print(media_df)

else:
    print(f"Error: {response.status_code} - {response.text}")


<Response [200]>
                  id                                            caption  \
0  17861666277364573  Get ready for the launch of our amazing MVP. T...   

                                            hashtags media_type  \
0  #makaw #makawai #polygon #marketing #sentiment...      IMAGE   

                                           media_url  \
0  https://scontent.cdninstagram.com/v/t51.75761-...   

                                  permalink                 timestamp  \
0  https://www.instagram.com/p/DG8Hi6vNu1K/  2025-03-08T13:03:26+0000   

   like_count latest_query_completed_timestamp  
0           1       2025-03-08T13:06:56.417750  


In [16]:
display(media_df)

,id,caption,hashtags,media_type,media_url,permalink,timestamp,like_count,latest_query_completed_timestamp
0,17861666277364573,Get ready for the launch of our amazing MVP. T...,#makaw #makawai #polygon #marketing #sentiment...,IMAGE,https://scontent.cdninstagram.com/v/t51.75761-...,https://www.instagram.com/p/DG8Hi6vNu1K/,2025-03-08T13:03:26+0000,1,2025-03-08T13:06:56.417750


In [28]:
#Store it in a database
import sqlite3

# Connect to (or create) the database
conn = sqlite3.connect("instagram_data.db")
cursor = conn.cursor()

# Create a table for Instagram posts (if not exists)
cursor.execute("""
CREATE TABLE IF NOT EXISTS instagram_posts (
    id TEXT PRIMARY KEY,
    caption TEXT,
    hashtags TEXT,
    media_type TEXT,
    media_url TEXT,
    permalink TEXT,
    timestamp TEXT,
    like_count INTEGER,
    api_completed_timestamp TEXT
)
""")

# Insert data into the database
def insert_into_db(data):
    with conn:
        cursor.executemany("""
        INSERT OR REPLACE INTO instagram_posts 
        (id, caption, hashtags, media_type, media_url, permalink, timestamp, like_count, api_completed_timestamp) 
        VALUES (?, ?, ?, ?, ?, ?, ?, ?, ?)
        """, data)

# Convert DataFrame to list of tuples and insert
media_df = pd.DataFrame(media_list)  # Assuming media_list from previous code
media_df["api_completed_timestamp"] = api_completed_timestamp  # Add timestamp
insert_into_db(media_df.to_records(index=False).tolist())

# Commit and close the connection
conn.commit()
conn.close()

OperationalError: table instagram_posts has no column named sentimentScore

In [19]:
#Query the data from the database
# Connect to the SQLite database
conn = sqlite3.connect('instagram_data.db')
cursor = conn.cursor()

# Query to fetch all posts from the instagram_posts table
cursor.execute(
    "SELECT id, hashtags,like_count FROM instagram_posts")

# Fetch all rows of the query result
rows = cursor.fetchall()

# Print each row in the result
for row in rows:
    print(row)

# Close the connection
conn.close()


('17920309653044896', '#makaw #makawai #testmakaw', 2)
('18034496246531367', '#makawai #makaw #testmakaw', 0)
('18090093601497188', '#makawai #makaw #testmakaw', 1)
('17861666277364573', '#makaw #makawai #polygon #marketing #sentimentanalysis #ai #blockchain', 1)


In [23]:
print(media_df.caption[0])

Get ready for the launch of our amazing MVP. The best and most user-engaging platform is right around the corner. Stay tuned for this incredible innovation which will revolutionize marketing as you know it!
